In [8]:
import cv2
import numpy as np
import csv
import os
import shutil
import tkinter as tk
from tkinter import filedialog
import pytesseract

def create_folders():
    origine_folder_path = r'C:\Users\anass\OneDrive\Desktop\projet-traitement-image\origine'
    processed_folder_path = r'C:\Users\anass\OneDrive\Desktop\projet-traitement-image\processed'
    
    if not os.path.exists(origine_folder_path):
        os.makedirs(origine_folder_path)
        print("Origine folder created successfully.")
    else:
        print("Origine folder already exists.")
    
    if not os.path.exists(processed_folder_path):
        os.makedirs(processed_folder_path)
        print("Processed folder created successfully.")
    else:
        print("Processed folder already exists.")

    return origine_folder_path, processed_folder_path

def detect_outline(image):
    try:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        contours = sorted(contours, key=cv2.contourArea, reverse=True)
        outline_contour = None
        for contour in contours:
            perimeter = cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, 0.02 * perimeter, True)
            if len(approx) == 4:
                outline_contour = approx
                break

        return outline_contour
    except Exception as e:
        print("Error:", e)
        return None

def detect_inner_lines(image, outline_contour):
    try:
        if outline_contour is not None:
            x, y, w, h = cv2.boundingRect(outline_contour)
            outline_region = image[y:y+h, x:x+w]

            gray = cv2.cvtColor(outline_region, cv2.COLOR_BGR2GRAY)

            thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 31, 5)

            contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)


            for contour in contours:
                if cv2.contourArea(contour) > 500:  
                    peri = cv2.arcLength(contour, True)
                    approx = cv2.approxPolyDP(contour, 0.02 * peri, True)
                    
                    if len(approx) == 4:
                        cv2.drawContours(outline_region, [approx], -1, (0, 255, 0), 2)
                        x, y, w, h = cv2.boundingRect(approx)
                        #cv2.line(outline_region, (x, y + h//2), (x + w, y + h//2), (0, 0, 255), 1)  # Horizontal line
                        #cv2.line(outline_region, (x + w//2, y), (x + w//2, y + h), (0, 0, 255), 1)  # Vertical line

            return outline_region
        else:
            print("Outline not found in the image.")
            return None
    except Exception as e:
        print("Error:", e)
        return None

def extract_table_from_image(image_path):
    try:
        origine_folder_path, processed_folder_path = create_folders()
        original_file_path = os.path.join(origine_folder_path, os.path.basename(image_path))
        shutil.copy(image_path, original_file_path)

        img = cv2.imread(image_path)

        outline_contour = detect_outline(img)

        inner_lines_image = detect_inner_lines(img, outline_contour)

        processed_image_path = os.path.join(processed_folder_path, os.path.splitext(os.path.basename(image_path))[0] + '_processed.jpg')
        cv2.imwrite(processed_image_path, inner_lines_image)

        return processed_image_path
    except Exception as e:
        print("Error:", e)
        return None, None

def import_file():
    try:
        file_path = filedialog.askopenfilename()
        if file_path:
            processed_image, csv_file_path = extract_table_from_image(file_path)
            if processed_image is not None and csv_file_path is not None:
                print('Processed image saved successfully:', processed_image)
        else:
            print('No file selected.')
    except Exception as e:
        print("Error:", e)

window = tk.Tk()
import_button = tk.Button(window, text='Import File', command=import_file)
import_button.pack()

window.mainloop()


Origine folder already exists.
Processed folder already exists.
Error: too many values to unpack (expected 2)
